In [ ]:
pip install pyserial

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Serial connection with Arduino for calibration

In [ ]:
import serial
import time

def calibrate(port, interval, batches):
    ser = serial.Serial(port, 9600, timeout=1)
    time.sleep(2)  # wait for Arduino reset

    command = f"{interval},{batches}\n"
    ser.write(command.encode())
    print(f"Sent: {command.strip()}")

    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            print(f"Arduino: {line}")
            if line == "Completed":
                break

    ser.close()

Sent: 500,4
Arduino: OK
Arduino: DONE


In [ ]:
# Execute calibration
# Replace 'COM3' with the correct port for your Arduino
# Adjust interval and batches as needed
calibrate("COM3", interval=500, batches=4)

# Scan command

In [7]:
import serial
import time

# Plate type definitions
plate_types = {
    "Thinfilm": {
        "offsetX": 15.0, "offsetY": 15.0, "colDist": 30.95, "rowDist": 27.95, "rows": 3, "cols": 4
    },
    "Corning_24": {
        "offsetX": 17.48, "offsetY": 13.80, "colDist": 19.30, "rowDist": 19.30, "rows": 4, "cols": 6
    },
    "Corning_48": {
        "offsetX": 18.16, "offsetY": 10.12, "colDist": 13.08, "rowDist": 13.08, "rows": 6, "cols": 8
    },
    "Corning_96": {
        "offsetX": 14.38, "offsetY": 11.23, "colDist": 9.00, "rowDist": 9.00, "rows": 8, "cols": 12
    }
    # Add more plate types as needed
}

def scan_plate(port, plate_type_name, wait_interval, start_pos, end_pos):
    if plate_type_name not in plate_types:
        print("Invalid plate type selected.")
        return

    plate_data = plate_types[plate_type_name]

    # Format: offsetX,offsetY,colDist,rowDist,rows,cols,wait,startPos,endPos
    command = f"scan_plate {plate_data['offsetX']},{plate_data['offsetY']},{plate_data['colDist']},{plate_data['rowDist']},{plate_data['rows']},{plate_data['cols']},{wait_interval},{start_pos},{end_pos}\n"

    ser = serial.Serial(port, 9600, timeout=2)
    time.sleep(2)  # Allow Arduino to reset

    ser.write(command.encode())

    print(f"Sent: {command.strip()}")
    print("Waiting for Arduino response:")

    while True:
        line = ser.readline().decode().strip()
        if line:
            print("Arduino:", line)
        if "Scan completed" in line:
            break

    ser.close()


In [10]:
# Execute scanning
# Replace 'COM3' with the correct port for your Arduino
# Adjust wait_interval, start_pos, and end_pos as needed
# start_pos and end_pos should be in the format "A1", "C3", etc
# They denote the top left and bottom right corners of the scanning area
scan_plate("COM3", "Corning_48", 100, "B1", "C3")

Sent: scan_plate 18.16,10.12,13.08,13.08,6,8,100,B1,C3
Waiting for Arduino response:
Arduino: Scan completed in 0 milliseconds.
